# Exemple de résolution du TP 1

## Problème

2. Si l’on considère un groupe de 4 personnes, combien y a-t-il de façon de les répartir
dans 2 équipes de 2 ?

### Produire une solution de 3 manières différentes :

1. De manière analytique (en utilisant des formules mathématiques).

1. En utilisant une méthode de force brute (en énumérant toutes les combinaisons possibles de manière séquentielle).

1. En utilisant une méthode probabiliste (en simulant un grand nombre de combinaisons de 4 lancers de dé de manière aléatoire).

### Hint

La bibliothèque standard de Python inclus un module très utile pour le dénombrement : [`itertools`](https://docs.python.org/fr/3/library/itertools.html)

En particulier les itérateurs combinatoires :

* `product()` : produit cartésien, équivalent à une boucle for imbriquée
* `permutations()`
* `combinations()`
* `combinations_with_replacement()`

## Code
### Import et configuration

In [3]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import comb, factorial as fac

In [6]:
%config InlineBackend.figure_format="retina"  # For high DPI display

sns.set_style("darkgrid")
sns.set_context("notebook")

### Helper functions

In [7]:
def print_pct(p, prec=1):
    print(f"{p:.{prec}%}")

### Solution 1 (analytique)

$$\frac{\displaystyle\binom{N}{n}}{2!}$$

In [8]:
N = 4
n = 2

comb(N, n) / fac(2)

np.float64(3.0)

### Solution 2 (force brute)

#### Définition des dès

In [9]:
PEOPLE = list("ABCD")
PEOPLE

['A', 'B', 'C', 'D']

In [10]:
people_df = pd.DataFrame(itertools.permutations(PEOPLE, 4))
people_df.columns = people_df.columns + 1
# df.columns = [f"p{i}" for i in df.columns + 1]
people_df

,1,2,3,4
0,A,B,C,D
1,A,B,D,C
2,A,C,B,D
3,A,C,D,B
4,A,D,B,C
5,A,D,C,B
6,B,A,C,D
7,B,A,D,C
8,B,C,A,D
9,B,C,D,A


In [11]:
people_df["g1"] = people_df.loc[:, 1:2].apply(frozenset, axis=1)
people_df["g2"] = people_df.loc[:, 3:4].apply(frozenset, axis=1)
people_df.drop(columns=range(1, 5), inplace=True)
people_df

,g1,g2
0,"(B, A)","(D, C)"
1,"(B, A)","(D, C)"
2,"(C, A)","(D, B)"
3,"(C, A)","(D, B)"
4,"(D, A)","(C, B)"
5,"(D, A)","(B, C)"
6,"(B, A)","(D, C)"
7,"(B, A)","(D, C)"
8,"(C, B)","(D, A)"
9,"(C, B)","(D, A)"


In [12]:
people_df.drop_duplicates(inplace=True)
people_df

,g1,g2
0,"(B, A)","(D, C)"
2,"(C, A)","(D, B)"
4,"(D, A)","(C, B)"
8,"(C, B)","(D, A)"
10,"(D, B)","(C, A)"
16,"(D, C)","(B, A)"


In [13]:
groups = people_df.apply(frozenset, axis=1)
groups

0     ((D, C), (B, A))
2     ((D, B), (C, A))
4     ((C, B), (D, A))
8     ((C, B), (D, A))
10    ((D, B), (C, A))
16    ((D, C), (B, A))
dtype: object

In [14]:
groups.drop_duplicates(inplace=True)
groups

0    ((D, C), (B, A))
2    ((D, B), (C, A))
4    ((C, B), (D, A))
dtype: object

In [15]:
def make_equal_groups(df: pd.DataFrame, group_n: int):
    df.columns = df.columns + 1
    people_n = len(df.columns)
    df["g1"] = df.loc[:, 1:group_n].apply(frozenset, axis=1)
    df["g2"] = df.loc[:, group_n + 1:people_n].apply(frozenset, axis=1)
    df.drop(columns=range(1, people_n + 1), inplace=True)
    df.drop_duplicates(inplace=True)
    groups = df.apply(frozenset, axis=1)
    groups.drop_duplicates(inplace=True)
    return groups.reset_index(drop=True)

In [16]:
ppl_df = pd.DataFrame(itertools.permutations(PEOPLE, 4))

make_equal_groups(ppl_df, group_n=2)

0    ((D, C), (B, A))
1    ((D, B), (C, A))
2    ((C, B), (D, A))
dtype: object

---

### Solution 3 (probabiliste)

La bibliothèque open source NumPy est destinée à manipuler des matrices ou tableaux multidimensionnels.
Elle propose également de nombreuses fonctions mathématiques et notemment des outils pour générer et manipuler des variables aléatoires.

Les versions récentes de NumPy favorise l'utilisation du générateur aléatoire par défaut comme expliqué dans le lien ci-dessous :

https://numpy.org/doc/stable/reference/random/generator.html

TLDR : 
```python
rng = np.random.default_rng(seed)
rng.integers()
# ou
rng.random()
# ou
rng.permutation()
# ...
```

In [17]:
SEED = 1234  # Pour la reproductibilité des résultats (valeur arbitraire)

RNG = np.random.default_rng(SEED)

In [18]:
def rdm_permutation(rng, people):
    return rng.permutation(people).tolist()

In [19]:
rdm_permutation(RNG, PEOPLE)

['A', 'D', 'B', 'C']

In [20]:
def rdm_perm_sim(rng, people, perm_n):
    return [rdm_permutation(rng, people) for _ in range(perm_n)]

In [21]:
SIM_N = 100
groups_sim_df = pd.DataFrame(rdm_perm_sim(RNG, PEOPLE, SIM_N))
groups_sim_df

,0,1,2,3
0,B,C,D,A
1,B,D,A,C
2,A,B,D,C
3,B,D,A,C
4,B,A,C,D
...,...,...,...,...
95,A,C,D,B
96,D,C,A,B
97,B,A,D,C
98,A,C,B,D


In [22]:
make_equal_groups(groups_sim_df, group_n=2)

0    ((C, B), (D, A))
1    ((D, B), (C, A))
2    ((D, C), (B, A))
dtype: object

### (Optionnel) Visualisation de la distribution

Distribution des sommes des lancers des 4 dés

In [23]:
# sns.histplot(rolls_sim_df, x="sums", discrete=True, stat="probability")
# plt.show()

TD 1 Analytique

In [24]:
N = 50
n= 3
somme = 1

for i in range (n) :
    #question b
    somme = somme *(N-i)
print(somme,"combinaisons possibles pour 3 roles différents 50 pers")
#/** question a **/
somme/=fac(n)
print(somme,"combinaisons possibles")

117600 combinaisons possibles pour 3 roles différents 50 pers
19600.0 combinaisons possibles


$$\frac{\displaystyle\binom{N}{n}}{2!}$$

Solution 1 (analytique) de la 6) a

In [25]:
import math

Nombre_de_cartes = 52
cartes_distribués = 2
nombre_as = 4
nombre_val10 = 16
combinaisons = math.comb(Nombre_de_cartes, cartes_distribués)
combi_de21 = nombre_as * nombre_val10
print(combi_de21 / combinaisons)



0.048265460030165915


---

Solution 1 (analytique) de la 6) b

In [26]:
#Valeur = 15 on cherche la probabilité que la prochaine carte fait perdre le joueur
cartes_restantes=50
nmbr_prochaines_cartesperd=27
tirage=1
chance_perte=27/50
print(chance_perte)

0.54


Solution 3 manière probabiliste 6.a)

In [41]:
import numpy as np
import pandas as pd

SIM_N = 1000
RNG = np.random.default_rng()


results = []
blackjack_count = 0
#Choix et stockage des cartes
for i in range(SIM_N):
    carte1 = RNG.choice([11] * 4 + [10] * 16 + list(range(2, 10)) * 4)
    carte2 = RNG.choice([11] * 4 + [10] * 16 + list(range(2, 10)) * 4)
    while carte1 == carte2:
        carte2 = RNG.choice([11] * 4 + [10] * 16 + list(range(2, 10)) * 4)

    somme = carte1 + carte2
    is_blackjack = (somme == 21)

    if is_blackjack:
        blackjack_count += 1

    # Stocker les résultats
    results.append({
        'Simulation': i + 1,
        'Carte 1': carte1,
        'Carte 2': carte2,
        'Somme': somme,
        'Blackjack': 'Oui' if is_blackjack else 'Non'
    })

# Création de tableau
results_df = pd.DataFrame(results)

# Résultats finaux
print("═" * 60)
print(f"RÉSULTATS DES {SIM_N} SIMULATIONS")
print("═" * 60)
print(f"Nombre total de blackjacks: {blackjack_count}")
print(f"Probabilité simulée: {blackjack_count/SIM_N:.4f} ({blackjack_count/SIM_N*100:.2f}%)")
print("═" * 60)

blackjacks_df = results_df[results_df['Blackjack'] == 'Oui']
if not blackjacks_df.empty:
    print(f"\nDétail des {len(blackjacks_df)} blackjacks obtenus:")
    print("─" * 40)
    print(blackjacks_df[['Simulation', 'Carte 1', 'Carte 2', 'Somme']].to_string(index=False))
else:
    print("\nAucun blackjack obtenu dans cette simulation.")

# Statistiques supplémentaires
print(f"\nSTATISTIQUES:")
print("─" * 20)
print(f"Total simulations: {SIM_N}")
print(f"Blackjacks: {blackjack_count}")
print(f"Non-blackjacks: {SIM_N - blackjack_count}")
print(f"Pourcentage de blackjacks: {blackjack_count/SIM_N*100:.2f}%")


print(f"\nRÉPARTITION DE Victoires:")
sum_stats = results_df['Somme'].value_counts().sort_index()
for somme, count in sum_stats.items():
    if somme == 21 :
        print("Resultats favorables: ",count,"victoires")

════════════════════════════════════════════════════════════
RÉSULTATS DES 1000 SIMULATIONS
════════════════════════════════════════════════════════════
Nombre total de blackjacks: 63
Probabilité simulée: 0.0630 (6.30%)
════════════════════════════════════════════════════════════

Détail des 63 blackjacks obtenus:
────────────────────────────────────────
 Simulation  Carte 1  Carte 2  Somme
          8       11       10     21
         12       11       10     21
         27       10       11     21
         48       11       10     21
         84       11       10     21
         87       11       10     21
        117       10       11     21
        130       11       10     21
        133       10       11     21
        135       11       10     21
        194       11       10     21
        211       10       11     21
        213       11       10     21
        240       11       10     21
        243       10       11     21
        262       10       11     21
        270   